In [ ]:
import os
import sys
import time

from biokb_chebi.db.importer import DatabaseImporter
from biokb_chebi.manager import MySQLDatabase
from biokb_chebi.neo4j_importer import Neo4jImporter
from biokb_chebi.rdf.turtle import TurtleCreator

sys.path.append(os.path.join("..", "tests"))

from docker_compose_defaults import (  # type: ignore
    MYSQL_DATABASE,
    MYSQL_HOST,
    MYSQL_PASSWORD,
    MYSQL_PORT,
    MYSQL_USER,
    NEO4J_PASSWD,
    NEO4J_URI,
    NEO4J_USER,
)

# Test package

In [ ]:
# create engine
mysql_db = MySQLDatabase(
    user=MYSQL_USER,
    password=MYSQL_PASSWORD,
    host=MYSQL_HOST,
    port=MYSQL_PORT,
    db_name=MYSQL_DATABASE,
)

start_and_stop_podman_compose = True

if start_and_stop_podman_compose:
    command = "podman-compose -f ../docker-compose.yml up -d mysql neo4j"
    os.popen(command).read()
    time.sleep(5)

## Import data into MySQL

In [ ]:
importer = DatabaseImporter(engine=mysql_db.engine)
importer.import_db()

## Create ttls

In [ ]:
turtle_creator = TurtleCreator(engine=mysql_db.engine)
turtle_creator.create_all_ttls()

## Import turtle files into Neo4J

In [ ]:
neo4j_importer = Neo4jImporter(
    neo4j_pwd=NEO4J_PASSWD,
    neo4j_user=NEO4J_USER,
    neo4j_uri=NEO4J_URI,
    delete_existing_graph=True,
)
neo4j_importer.import_ttls()

In [ ]:
if start_and_stop_podman_compose:
    command = "podman-compose -f ../docker-compose.yml stop mysql neo4j"
    os.popen(command).read()

## Check the imports

1. Start MySQL, Neo4J and phpMyAdmin with `podman-compose up -d` from the project root folder in the terminal
2. Open [PhpMyAdmin](http://localhost:8081) with
   1. *user*: biokb_chebi_user
   2. *password*: biokb_chebi_password
3. Open [Neo4J Browser](http://localhost:7475/browser/) and login with ...
   - *Connect URL*: neo4j://localhost:7688
   - *Authentication type*: Username/Password
   - *Username*: neo4j
   - *Password*: neo4j_password